### Import packages correction

In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import tensorflow as tf

# Get your datasets

In [2]:
from data.dataset import H5PY_RW
from data.data_tools import read_data_header, males_keys, females_keys

H5_dic = read_data_header()
chunk_size = 512*30

males = H5PY_RW().open_h5_dataset('test_raw.h5py', subset = males_keys(H5_dic)).set_chunk(chunk_size).shuffle()
fem = H5PY_RW().open_h5_dataset('test_raw.h5py', subset = females_keys(H5_dic)).set_chunk(chunk_size).shuffle()
print 'Data with', len(H5_dic), 'male and female speakers'


Data with 40 male and female speakers


# Mixing the dataset

In [3]:
from data.dataset import Mixer

mixed_data = Mixer([males, fem], with_mask=False, with_inputs=True)

# Training set selection
mixed_data.select_split(0)

# Model pretrained loading

In [4]:
learning_rate = 0.0001
batch_size = 1
N = 256
max_pool = 256
regularization = 0.0001

config_model = {}
config_model["alpha"] = learning_rate
config_model["reg"] = regularization
config_model["batch_size"] = batch_size
config_model["chunk_size"] = chunk_size
config_model["N"] = N
config_model["maxpool"] = max_pool
config_model["type"] = "pretraining"
config_model["smooth_size"] = 10
idd = ''.join('-{}={}-'.format(key, val) for key, val in sorted(config_model.items()))
config_model["type"] = "DPCL_train_front"

In [5]:
from models.adapt import Adapt

full_id = 'flat-limit-0477'+idd

folder='DPCL_train_front'
model = Adapt(config_model=config_model,pretraining= False, folder=folder)
model.create_saver()
model.restore_model('pretraining', full_id)

ID : AdaptiveNet-shiny-frog-0617
INFO:tensorflow:Restoring parameters from /home/anthony/das/log/pretraining/AdaptiveNet-flat-limit-0477-N=256--alpha=0.0001--batch_size=1--chunk_size=15360--maxpool=256--reg=0.0001--smooth_size=10--type=pretraining-/model.ckpt


## Connect DPCL model to the front end

In [6]:
from models.dpcl import DPCL

with model.graph.as_default():
    model.connect_front(DPCL)
    model.sepNet.output = model.sepNet.prediction
    model.cost = model.sepNet.cost
    model.optimize
    model.freeze_front()
    model.tensorboard_init()

Tensor("prediction/l2_normalize:0", shape=(?, ?, 256, 40), dtype=float32)
[<tf.Variable 'conv/W:0' shape=(1, 1024, 1, 256) dtype=float32_ref>, <tf.Variable 'smooth/smoothing_filter:0' shape=(1, 10, 1, 1) dtype=float32_ref>, <tf.Variable 'prediction/W:0' shape=(1, 600, 10240) dtype=float32_ref>, <tf.Variable 'prediction/b:0' shape=(10240,) dtype=float32_ref>, <tf.Variable 'prediction/forward_BLSTM_1/rnn/basic_lstm_cell/kernel:0' shape=(556, 1200) dtype=float32_ref>, <tf.Variable 'prediction/forward_BLSTM_1/rnn/basic_lstm_cell/bias:0' shape=(1200,) dtype=float32_ref>, <tf.Variable 'prediction/backward_BLSTM_1/rnn/basic_lstm_cell/kernel:0' shape=(556, 1200) dtype=float32_ref>, <tf.Variable 'prediction/backward_BLSTM_1/rnn/basic_lstm_cell/bias:0' shape=(1200,) dtype=float32_ref>, <tf.Variable 'prediction/forward_BLSTM_2/rnn/basic_lstm_cell/kernel:0' shape=(900, 1200) dtype=float32_ref>, <tf.Variable 'prediction/forward_BLSTM_2/rnn/basic_lstm_cell/bias:0' shape=(1200,) dtype=float32_ref>, <

In [7]:
with model.graph.as_default():
    uninit =[v for v in tf.global_variables() if v.name.split(':')[0] in set(model.sess.run(tf.report_uninitialized_variables()))]
    print uninit
    init = tf.variables_initializer(uninit)

[<tf.Variable 'prediction/W:0' shape=(1, 600, 10240) dtype=float32_ref>, <tf.Variable 'prediction/b:0' shape=(10240,) dtype=float32_ref>, <tf.Variable 'prediction/forward_BLSTM_1/rnn/basic_lstm_cell/kernel:0' shape=(556, 1200) dtype=float32_ref>, <tf.Variable 'prediction/forward_BLSTM_1/rnn/basic_lstm_cell/bias:0' shape=(1200,) dtype=float32_ref>, <tf.Variable 'prediction/backward_BLSTM_1/rnn/basic_lstm_cell/kernel:0' shape=(556, 1200) dtype=float32_ref>, <tf.Variable 'prediction/backward_BLSTM_1/rnn/basic_lstm_cell/bias:0' shape=(1200,) dtype=float32_ref>, <tf.Variable 'prediction/forward_BLSTM_2/rnn/basic_lstm_cell/kernel:0' shape=(900, 1200) dtype=float32_ref>, <tf.Variable 'prediction/forward_BLSTM_2/rnn/basic_lstm_cell/bias:0' shape=(1200,) dtype=float32_ref>, <tf.Variable 'prediction/backward_BLSTM_2/rnn/basic_lstm_cell/kernel:0' shape=(900, 1200) dtype=float32_ref>, <tf.Variable 'prediction/backward_BLSTM_2/rnn/basic_lstm_cell/bias:0' shape=(1200,) dtype=float32_ref>, <tf.Variab

# Model creation

# Pretraining the model 

In [8]:
nb_iterations = 1000

#initialize the model
model.sess.run(init)

for i in range(nb_iterations):
    X_in, X_mix, Ind = mixed_data.get_batch(batch_size)
    c = model.train(X_mix, X_in, learning_rate, i)
    print 'Step #'  ,i,' loss=', c 

    if i%20 == 0: #cost_valid < cost_valid_min:
        print 'DAS model saved at iteration number ', i,' with cost = ', c 
        model.save(i)

Step # 0  loss= 137.988
DAS model saved at iteration number  0  with cost =  137.988
Step # 1  loss= 127.968
Step # 2  loss= 119.8
Step # 3  loss= 114.804
Step # 4  loss= 110.379
Step # 5  loss= 107.097
Step # 6  loss= 104.243
Step # 7  loss= 100.086
Step # 8  loss= 100.534
Step # 9  loss= 98.9576
Step # 10  loss= 96.7935
Step # 11  loss= 96.4269
Step # 12  loss= 95.1378
Step # 13  loss= 94.222
Step # 14  loss= 93.127
Step # 15  loss= 92.7828
Step # 16  loss= 92.2283
Step # 17  loss= 91.6716
Step # 18  loss= 90.722
Step # 19  loss= 90.8449
Step # 20  loss= 90.4047
DAS model saved at iteration number  20  with cost =  90.4047
Step # 21  loss= 89.5077
Step # 22  loss= 89.4204
Step # 23  loss= 89.4762
Step # 24  loss= 89.2782
Step # 25  loss= 88.8426
Step # 26  loss= 87.8713
Step # 27  loss= 88.3852
Step # 28  loss= 88.4611
Step # 29  loss= 88.1511
Step # 30  loss= 87.9636
Step # 31  loss= 87.4099
Step # 32  loss= 84.8185
Step # 33  loss= 87.5965
Step # 34  loss= 86.9726
Step # 35  loss= 

KeyboardInterrupt: 